# 📊 **QA Script for Population Projections**

## 📝 **Introduction**
This notebook aims to identify outliers in the GLA population projection data. The analysis involves loading the dataset, preprocessing the data, defining utility functions, performing outlier detection, and presenting the results through visualisations and a 
HMTL report.

### 🎯 **Goals**
The analysis will focus on the following objectives:
- **Load and preprocess** the population projections dataset.
- Define utility functions.
- Perform **basic checks** on the dataset:
  - Range of years covered.
  - Missing values.
  - Duplicates.
  - Descriptive statistics.
  - Breakdown by components.
  - Detecting negative values.
  - Age group ranges.
- **Outlier Detection** over time for each component:
  - Identify outliers using **Z-scores** and **Robust Z-scores**.
  - Analyse by **component**, **ward**, and **borough**.
  - Handle **infinite values** separately.
- **Total Population Outliers**:
  - Use Z-scores and Robust Z-scores for comparison.
  - Perform **cross-sectional comparisons**: Examine changes between boroughs and wards for a given year.
  - Conduct **temporal comparisons**: Measure percentage changes between years for both boroughs and wards.
  - Handle **infinite values** separately.
- **Gender Outliers**:
  - Investigate abnormal **gender ratios**.
  - Analyse by component.
  - Adjust the **outlier standard deviation thresholds** as needed based on different components.
- **Key Visualisations**:
  - Display the distribution of components.
  - Group data by **age ranges**.
  - Visualise **yearly totals**.
  - Show yearly total trends over time, broken down by components.
  - Create **population pyramids**.
- **Collate Outliers**: Summarise and determine the key outlier rows.

---

## 📂 **Datasets**
How should the dataset be structured?

### Dataset 1
The functions are designed to take datasets in the form produced by the GLA Population Projection Workflow. In the current iteration of this workflow, the first dataset contains several key columns and components as outlined below:

#### Main Columns
- **gss_code**: Borough geocode (e.g., GSS code).
- **la_name**: Local Authority name.
- **Year**: The year of the population data.
- **Sex**: The gender (male/female).
- **Age**: The age group or specific age.
- **Value**: Count of the population in the given category.
- **gss_code_ward**: Geocode for the ward.
- **ward_name**: Name of the ward.

#### Components Column:
This column includes specific population-related metrics:
- **net-flow**: Population migration inflow minus migration outflow.
- **popn**: Total population.
- **birth**: Number of births.
- **deaths**: Number of deaths.

### Dataset 2

The second dataset that is examined for outliers is the average household size (ahs) dataset. This is analysed using the find_ahs_outliers_with_context function.

The dataset just contains average household size from 2021 until 2050 by ward

### 


---

## 🛠️ **Structure**
1. [**Load Data** the population projections dataset.](#load-data)
2. [**Define Utility Functions** for effective use.](#define-utility-functions)
3. [**Process Data**](#process-data)
4. [**Basic Checks** on the dataset.](#basic-checks)
5. [**Population Consistency Over Time** for each component.](#population-consistency-over-time)
6. [**Total Population Outliers**](#total-population-outliers)
7. [**Gender Outliers**](#gender-outliers)
8. [**Key Visualisations**](#key-visualisations)
9. [**Collate Outliers** to determine key outlier rows.](#collate-outliers)
10. [**Average Household Size** Outliers.](#average-household-size_outliers)
11. [**HTML** Report.](#produce-html-report)



## Load Data
This section will cover how to load and preprocess the dataset.

---


In [ ]:
import pandas as pd
import numpy as np
import pyreadr
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from typing import Dict
from scipy.stats import zscore
from scipy.stats import skew
import plotly.express as px
import plotly.io as pio
import dash_bootstrap_components as dbc


In [ ]:
os.chdir(r'C:\Users\user\Documents\population_data\combined_10yr_central_fert')
combined_10yr_fert = pd.read_csv('combined_10yr_central_fert.csv').iloc[:, 1:]

In [ ]:
combined_10yr_fert

## Define Utility Functions
Define utility functions that will be used for various parts of the analysis.

---

In [ ]:
def view_descriptive_statistics(df, columns):
    """
    Calculate descriptive statistics, including mean, median, and mode, for specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns (list): List of columns for which to calculate the statistics.
    
    Returns:
    pd.DataFrame: DataFrame containing the descriptive statistics including median and mode.
    """
    # Get descriptive statistics using describe()
    descriptive_stats = df[columns].describe()

    # Calculate median for each column
    median = df[columns].median()

    # Calculate mode for each column (in case of multiple modes, take the first one)
    mode = df[columns].mode().iloc[0]

    # Add median and mode to the descriptive statistics DataFrame
    descriptive_stats.loc['median'] = median
    descriptive_stats.loc['mode'] = mode

    # Return the combined descriptive statistics
    return descriptive_stats

In [ ]:
def create_age_bins(df, age_column='age', bins=None, labels=None):
    """
    Create age bins for the specified age column in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the age data.
    age_column (str): The name of the column containing age data. Default is 'age'.
    bins (list): A list of bin edges for categorizing ages. Default is None.
    labels (list): A list of labels for the bins. Default is None.

    Returns:
    pd.DataFrame: The DataFrame with a new 'age' column containing binned age data.
    """
    
    # If bins and labels are not provided, set default values
    if bins is None:
        bins = [-1, 18, 30, 40, 50, 60, 70, 80, 89, 90]
    
    if labels is None:
        labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-89', '90+']
    
    # Create a copy of the original DataFrame to avoid modifying it directly
    binned_df = df.copy()
    
    # Create age bins
    binned_df[age_column] = pd.cut(binned_df[age_column], bins=bins, labels=labels)
    
    return binned_df

# Example usage:
# combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)


In [ ]:
def calculate_zscores_and_find_outliers(df, component_columns, handle_inf=True, Geography='borough', z_score_threshold=2, For_population_totals=False, population_analysis_type='cross-sectional'):
    """
    Computes z-scores or robust z-scores (depending on distribution) for the respective columns,
    and returns DataFrames containing outliers for either the component columns (handle_inf=True) 
    or the percentage change columns (handle_inf=False).

    Parameters:
    df (pd.DataFrame): The input DataFrame containing population data component value columns.
    component_columns (str or list): A single column name or a list of column names to be analysed.
    handle_inf (bool): If True, uses the component columns to determine outliers. 
                       If False, uses percentage change columns to determine outliers.
    Geography (str): Specifies whether to group by 'borough' (using 'gss_code') or 'ward' (using 'gss_code_ward').
                     Default is 'borough'.
    For_population_totals (bool): If True, calculates total population sums and percentage changes before proceeding 
                                  to z-score and outlier analysis.
    population_analysis_type (str): Specifies whether to do 'cross-sectional' or 'temporal' analysis for population totals.
                                    Default is 'cross-sectional'.
    z_score_threshold (float or int): The threshold to consider as an outlier based on the z-score. Default is 2.

    Returns:
    dict: A dictionary containing DataFrames with outliers for each respective column based on the z-score threshold.
    """
    
    # If a single column name is provided as a string, convert it to a list
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}
    z_score_type = {}  # Dictionary to store which method was used

    # Grouping and pivot based on the Geography parameter
    if Geography == 'borough':
        geo_column = 'gss_code'
    elif Geography == 'ward':
        geo_column = 'gss_code_ward'
    else:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    # Automatically create pct_change_columns based on component_columns
    pct_change_columns = [f"{col}_pct_change" for col in component_columns]

    #Calculate the percentage change for the component columns
    df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
    #pivoted = df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()

    # If For_population_totals is True, calculate population totals and percentage changes
    if For_population_totals:
        # Filter the DataFrame to only keep rows where the 'component' is 'popn'
        # popn_df = pivoted[pivoted['component'] == 'popn']
        
        # Pivot the data to get 'popn' values by geography, sex, age, and year
        # pivoted = popn_df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()
        
        # Ensure that there's a 'popn' column in the resulting DataFrame
        if 'popn' in df.columns:
            df['popn'] = df['popn']  # Extract the 'popn' column
        else:
            raise ValueError("The 'popn' column is missing after pivoting.")

        # Group by geography and year, and sum the population values, create total population dfs for crossectional and temporal analysis change
        population_sum = df.groupby([geo_column, 'year'])['popn'].sum().reset_index()

        population_sum_time = population_sum.copy()
        population_sum_cross = population_sum.copy()

        if population_analysis_type == 'temporal':
            # Temporally: Calculate the population change over the years for each gss_code or ward
            population_sum_time['popn_pct_change_temporal'] = population_sum_time.groupby(geo_column)['popn'].pct_change() * 100
        elif population_analysis_type == 'cross-sectional':
            # Cross-sectionally: Compare the population between different gss_code or wards for the same year and compare to the mean
            population_sum_cross['popn_mean'] = population_sum_cross.groupby('year')['popn'].transform('mean')
            population_sum_cross['popn_pct_change_cross'] = ((population_sum_cross['popn'] - population_sum_cross['popn_mean']) / population_sum_cross['popn_mean']) * 100
        else:
            raise ValueError("population_analysis_type must be either 'cross-sectional' or 'temporal'.")

    # Now, decide how to determine outliers based on handle_inf (handling the infinite values e.i. no percentage change value) and pct_change_columns
    if handle_inf:
        # Outliers based on component columns value (not percentage change)
        for comp_col, pct_change_col in zip(component_columns, pct_change_columns):
            if pct_change_col in df.columns:
                # Filter rows where percentage change columns have inf or -inf
                df_filtered = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[comp_col, pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[comp_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[comp_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[comp_col] = 'Normal Z-Score'
                        print(f"{comp_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[comp_col].median()
                        mad = stats.median_abs_deviation(df_filtered[comp_col])
                        df_filtered['robust_z_score'] = (df_filtered[comp_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[comp_col] = 'Robust Z-Score'
                        print(f"{comp_col} used Robust Z-Score.")

                    # Store the outliers for this component column
                    outliers_dict[comp_col] = outliers
                else:
                    outliers_dict[comp_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{comp_col} does not exist in DataFrame")

    else:
        # Outliers based on pct_change_columns
        for pct_change_col in pct_change_columns:
            # For population totals choose the correct DataFrame (from above) based on the selected column
            if population_analysis_type == 'temporal' and 'popn_pct_change_temporal' in pct_change_col:
                df_filtered = population_sum_time  # Use the temporal population data
            elif population_analysis_type == 'cross-sectional' and 'popn_pct_change_cross' in pct_change_col:
                df_filtered = population_sum_cross  # Use the cross-sectional population data
            else:
                df_filtered = df  # Default to the original df if other percentage columns are provided

            #pct column for component column zscores
            if pct_change_col in df_filtered.columns:
                # Replace inf and -inf with NaN and work on the entire DataFrame after cleaning
                df_filtered = df_filtered.replace([np.inf, -np.inf], np.nan).dropna(subset=[pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[pct_change_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[pct_change_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[pct_change_col] = 'Normal Z-Score'
                        print(f"{pct_change_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[pct_change_col].median()
                        mad = stats.median_abs_deviation(df_filtered[pct_change_col])
                        df_filtered['robust_z_score'] = (df_filtered[pct_change_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[pct_change_col] = 'Robust Z-Score'
                        print(f"{pct_change_col} used Robust Z-Score.")

                    # Store the outliers for this percentage change column
                    outliers_dict[pct_change_col] = outliers
                else:
                    outliers_dict[pct_change_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{pct_change_col} does not exist in DataFrame")

    # Sort each DataFrame in outliers_dict by 'z_score' or 'robust_z_score' in descending order in one line
    for key in outliers_dict.keys():
        outliers_dict[key] = outliers_dict[key].sort_values(by='z_score' if 'z_score' in outliers_dict[key].columns else 'robust_z_score', ascending=False)

    return outliers_dict  # Returning z_score_type for further use if needed


In [ ]:
def calculate_zscores_and_find_outliers(df, component_columns, handle_inf=True, Geography='borough', z_score_threshold=2,
                                        For_population_totals=False, population_analysis_type='cross-sectional'):
    """
    Computes z-scores or robust z-scores for the specified columns and identifies outliers based on the provided criteria.
    """

    # Guard clauses to ensure valid inputs
    if not component_columns:
        raise ValueError("component_columns cannot be empty.")
    if z_score_threshold < 0:
        raise ValueError("z_score_threshold must be non-negative.")

    # Convert single column name to a list if necessary
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}
    z_score_type = {}  # Track z-score type for each column

    # Choose geographical grouping based on 'Geography' parameter
    geo_column = 'gss_code' if Geography == 'borough' else 'gss_code_ward' if Geography == 'ward' else None
    if not geo_column:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    pct_change_columns = [f"{col}_pct_change" for col in component_columns]
    df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()

    # Helper function for z-score calculation based on skewness
    def calculate_outliers(column, data):
        skewness = data[column].skew()
        if abs(skewness) < 0.5:  # Normal distribution
            data['z_score'] = stats.zscore(data[column])
            outliers = data[data['z_score'].abs() > z_score_threshold]
            z_score_type[column] = 'Normal Z-Score'
        else:  # Non-normal distribution: Robust Z-score
            median, mad = data[column].median(), stats.median_abs_deviation(data[column])
            data['robust_z_score'] = (data[column] - median) / mad
            outliers = data[data['robust_z_score'].abs() > z_score_threshold]
            z_score_type[column] = 'Robust Z-Score'
        return outliers

    if For_population_totals:
        if 'popn' not in df.columns:
            raise ValueError("The 'popn' column is required for population totals analysis.")
        
        # Process population totals for temporal or cross-sectional analysis
        population_sum = df.groupby([geo_column, 'year'])['popn'].sum().reset_index()
        population_sum_time, population_sum_cross = population_sum.copy(), population_sum.copy()
        
        if population_analysis_type == 'temporal':
            population_sum_time['popn_pct_change_temporal'] = population_sum_time.groupby(geo_column)['popn'].pct_change() * 100
        elif population_analysis_type == 'cross-sectional':
            population_sum_cross['popn_mean'] = population_sum_cross.groupby('year')['popn'].transform('mean')
            population_sum_cross['popn_pct_change_cross'] = ((population_sum_cross['popn'] - population_sum_cross['popn_mean']) / 
                                                             population_sum_cross['popn_mean']) * 100
        else:
            raise ValueError("population_analysis_type must be either 'cross-sectional' or 'temporal'.")

    # Outlier detection based on handle_inf flag
    if handle_inf:
        # Based on component columns
        for comp_col in component_columns:
            if comp_col in df.columns:
                df_filtered = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[comp_col])
                if not df_filtered.empty:
                    outliers_dict[comp_col] = calculate_outliers(comp_col, df_filtered)
    else:
        # Based on pct_change columns
        for pct_change_col in pct_change_columns:
            df_filtered = (population_sum_time if population_analysis_type == 'temporal' and 'popn_pct_change_temporal' in pct_change_col
                           else population_sum_cross if population_analysis_type == 'cross-sectional' and 'popn_pct_change_cross' in pct_change_col
                           else df.replace([np.inf, -np.inf], np.nan).dropna(subset=[pct_change_col]))

            if not df_filtered.empty:
                outliers_dict[pct_change_col] = calculate_outliers(pct_change_col, df_filtered)

    # Sort each DataFrame in outliers_dict by the z-score type
    for key, df_outliers in outliers_dict.items():
        z_type_column = 'z_score' if 'z_score' in df_outliers.columns else 'robust_z_score'
        outliers_dict[key] = df_outliers.sort_values(by=z_type_column, ascending=False)

    return outliers_dict

In [ ]:
def calculate_zscores_and_find_outliers_percentage_change(df, component_columns, Geography='borough', percentage_change_threshold=0.05, For_population_totals=False, population_analysis_type='cross-sectional'):
    """
    Computes outliers based on absolute percentage change for the specified columns, without handling infinite values.
    Outliers are detected using a percentage change threshold instead of z-scores.
    Optionally includes population totals analysis for either temporal or cross-sectional data.
    """
    # Guard clauses to ensure valid inputs
    if not component_columns:
        raise ValueError("component_columns cannot be empty.")
    if percentage_change_threshold < 0:
        raise ValueError("percentage_change_threshold must be non-negative.")
    if For_population_totals and 'popn' not in df.columns:
        raise ValueError("The 'popn' column is required for population totals analysis.")

    # Convert single column name to a list if necessary
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}

    # Choose geographical grouping based on 'Geography' parameter
    geo_column = 'gss_code' if Geography == 'borough' else 'gss_code_ward' if Geography == 'ward' else None
    if not geo_column:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    # Calculate absolute percentage change for each component column
    percentage_change_columns = [f"{col}_percentage_change" for col in component_columns]
    df[percentage_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()

    # If For_population_totals is True, calculate population-based percentage change
    if For_population_totals:
        population_sum = df.groupby([geo_column, 'year'])['popn'].sum().reset_index()
        population_sum_time, population_sum_cross = population_sum.copy(), population_sum.copy()

        if population_analysis_type == 'temporal':
            population_sum_time['popn_pct_change_temporal'] = population_sum_time.groupby(geo_column)['popn'].pct_change().abs() * 100
        elif population_analysis_type == 'cross-sectional':
            population_sum_cross['popn_mean'] = population_sum_cross.groupby('year')['popn'].transform('mean')
            population_sum_cross['popn_pct_change_cross'] = ((population_sum_cross['popn'] - population_sum_cross['popn_mean']).abs() / population_sum_cross['popn_mean']) * 100
        else:
            raise ValueError("population_analysis_type must be either 'cross-sectional' or 'temporal'.")

    # Outlier detection based on absolute percentage change threshold
    for pct_change_col in percentage_change_columns:
        df_filtered = df.dropna(subset=[pct_change_col])  # Dropping NaNs before analysis

        if not df_filtered.empty:
            # Detect outliers based on the absolute percentage change threshold
            outliers = df_filtered[df_filtered[pct_change_col] > percentage_change_threshold]

            if not outliers.empty:
                outliers_dict[pct_change_col] = outliers

    # If For_population_totals is True, include population-based outliers
    if For_population_totals:
        # Process population percentage change columns
        popn_pct_change_columns = []
        if population_analysis_type == 'temporal':
            popn_pct_change_columns.append('popn_pct_change_temporal')
        elif population_analysis_type == 'cross-sectional':
            popn_pct_change_columns.append('popn_pct_change_cross')

        for popn_pct_change_col in popn_pct_change_columns:
            df_filtered = df.dropna(subset=[popn_pct_change_col])  # Dropping NaNs before analysis

            if not df_filtered.empty:
                # Detect population outliers based on the absolute percentage change threshold
                outliers = df_filtered[df_filtered[popn_pct_change_col] > percentage_change_threshold]

                if not outliers.empty:
                    outliers_dict[popn_pct_change_col] = outliers

    # Sort each DataFrame in outliers_dict by percentage change
    for key, df_outliers in outliers_dict.items():
        outliers_dict[key] = df_outliers.sort_values(by=key, ascending=False)

    return outliers_dict


def calculate_zscores_and_find_outliers_value(df, component_columns, Geography='borough', z_score_threshold=2):
    """
    Computes z-scores or robust z-scores for the specified columns and identifies outliers,
    this function is used to handle values that are infinite, where handling infinite values such as where values change from 0 or to 0. 
    Thus percnetage change is not applicable in this case, so the values are used directly.
   
    """
    # Guard clauses to ensure valid inputs
    if not component_columns:
        raise ValueError("component_columns cannot be empty.")
    if z_score_threshold < 0:
        raise ValueError("z_score_threshold must be non-negative.")

    # Convert single column name to a list if necessary
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}
    z_score_type = {}  # Track z-score type for each column

    # Choose geographical grouping based on 'Geography' parameter
    geo_column = 'gss_code' if Geography == 'borough' else 'gss_code_ward' if Geography == 'ward' else None
    if not geo_column:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    percentage_change_columns = [f"{col}_percentage_change" for col in component_columns]
    df[percentage_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()

    # Helper function for z-score calculation based on skewness
    def calculate_outliers(column, data):
        skewness = data[column].skew()
        if abs(skewness) < 0.5:  # Normal distribution
            data['z_score'] = stats.zscore(data[column])
            outliers = data[data['z_score'].abs() > z_score_threshold]
            z_score_type[column] = 'Normal Z-Score'
        else:  # Non-normal distribution: Robust Z-score
            median, mad = data[column].median(), stats.median_abs_deviation(data[column])
            data['robust_z_score'] = (data[column] - median) / mad
            outliers = data[data['robust_z_score'].abs() > z_score_threshold]
            z_score_type[column] = 'Robust Z-Score'
        return outliers

    # Outlier detection based on handling inf
    for comp_col in component_columns:
        if comp_col in df.columns:
            df_filtered = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[comp_col]) #replacing inf values with NaN before proceeding
            if not df_filtered.empty:
                outliers_dict[comp_col] = calculate_outliers(comp_col, df_filtered)

    # Sort each DataFrame in outliers_dict by the z-score type
    for key, df_outliers in outliers_dict.items():
        z_type_column = 'z_score' if 'z_score' in df_outliers.columns else 'robust_z_score'
        outliers_dict[key] = df_outliers.sort_values(by=z_type_column, ascending=False)

    return outliers_dict

In [ ]:
def robust_z_score(series):
    """Calculate robust z-score using median and MAD."""
    median = series.median()
    mad = (series - median).abs().median()
    return 0.6745 * (series - median) / mad if mad != 0 else np.zeros_like(series)

def gender_outliers(df, component_columns, geography='borough', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5}):
    """
    Processes gender data for either wards or boroughs, and calculates outliers for specified components.
    Uses either z-score or robust z-score based on skewness.

    Parameters:
    - df: pandas DataFrame containing the raw data
    - component_columns: list or single component column name(s) for which ratios and outliers need to be calculated
    - geography: str, either 'ward' or 'borough', default is 'borough'
    - outlier_std: dict specifying how many standard deviations to use for each component's threshold calculation.
    
    Returns:
    - outliers_dict: dictionary of outlier DataFrames for each component
    """

    # Check geography type and set index columns accordingly
    if geography == 'ward':
        geo_col = 'gss_code_ward'
    else:
        geo_col = 'gss_code'
    
    # Step 1: Create the pivot table
    gender_pivot = df.pivot_table(
        index=[geo_col, 'year', 'age', 'component'],  # Geography column and other grouping columns
        columns='sex',                                # Columns for sex (male, female)
        values='value',                               # Values (count of males and females)
        aggfunc='sum'                                 # Aggregation function (sum)
    ).reset_index()
    
    # Step 2: Calculate the ratio of females to males
    gender_pivot['ratio_female_to_male'] = gender_pivot['female'] / gender_pivot['male']
    
    # Handle division by zero and missing values
    gender_pivot['ratio_female_to_male'].replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    gender_pivot['ratio_female_to_male'].fillna(np.nan, inplace=True)
    
    # Step 3: Pivot again to spread component values into columns
    gender_pivot = gender_pivot.pivot(
        index=[geo_col, 'year', 'age'], 
        columns='component', 
        values='ratio_female_to_male'
    ).reset_index()
    
    # Step 4: Convert the specified component columns to numeric
    for component in component_columns:
        gender_pivot[component] = pd.to_numeric(gender_pivot[component], errors='coerce')
    
    # Step 5: Detect skewness and calculate either z-score or robust z-score for each component
    outliers_dict = {}
    for component in component_columns:
        component_data = gender_pivot[component].dropna()

        # Calculate skewness
        component_skewness = skew(component_data)
        
        # Decide whether to use z-score or robust z-score
        if abs(component_skewness) > 0.5:
            # Use robust z-score if skewness is high
            z_column = 'robust_z_score'
            gender_pivot[z_column] = robust_z_score(gender_pivot[component])
        else:
            # Use standard z-score if skewness is low
            z_column = 'z_score'
            gender_pivot[z_column] = zscore(gender_pivot[component], nan_policy='omit')
        
        # Set the outlier threshold based on user input (default 2, 5, etc. depending on the component)
        threshold = outlier_std.get(component, 2)
        
        # Step 6: Identify outliers where absolute z-score is greater than the threshold
        outliers = gender_pivot[gender_pivot[z_column].abs() > threshold]
        
        # Step 7: Keep only relevant columns in the output (geo_col, year, age, z_score, and the component itself)
        outliers = outliers[[geo_col, 'year', 'age', component, z_column]]
        
        # Store the outliers DataFrame in the outliers dictionary
        outliers_dict[component] = outliers.reset_index(drop=True)
    
    return outliers_dict



In [ ]:
def tally_outliers(outlier_dfs: Dict[str, pd.DataFrame], 
                   tally_by_age: bool = False, 
                   geography: str = 'ward') -> pd.DataFrame:
    """
    Tally occurrences of the same 'gss_code_ward' or 'gss_code', 
    'year', and optionally 'age' across multiple DataFrames, 
    and merge with the total outlier score from 'robust_z_score' or 'z_score'.

    Parameters:
    ----------
    outlier_dfs : Dict[str, pd.DataFrame]
        A dictionary where keys are DataFrame names and values are the DataFrames themselves.
        Each DataFrame should contain 'gss_code_ward' or 'gss_code', 'year', and optionally 'age' columns.

    tally_by_age : bool, optional
        If True, the function will tally occurrences based on 'gss_code_ward' or 'gss_code', 'year', and 'age'.
        If False, it will tally based only on 'gss_code_ward' or 'gss_code' and 'year'. Default is False.

    geography : str, optional
        Specifies the geographical code to use. Set to 'borough' to use 'gss_code' 
        or 'ward' to use 'gss_code_ward'. Default is 'ward'.

    Returns:
    -------
    pd.DataFrame
        A DataFrame with counts of occurrences for each combination of 
        'gss_code_ward' or 'gss_code', 'year', and 'age' (if applicable), 
        along with a total count across all DataFrames and the total outlier score.
    """
    
    # Validate input
    if not isinstance(outlier_dfs, dict):
        raise ValueError("Expected 'outlier_dfs' to be a dictionary of DataFrames.")
    
    if geography not in ['borough', 'ward']:
        raise ValueError("geography must be either 'borough' or 'ward'.")

    # Initialise an empty DataFrame for the combined tally
    combined_tally = pd.DataFrame()

    # Define the geography column based on the geography argument
    geo_column = 'gss_code' if geography == 'borough' else 'gss_code_ward'

    # Loop through each DataFrame in the outlier_dfs dictionary
    for df_name, df in outlier_dfs.items():
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"Expected '{df_name}' to be a pandas DataFrame.")
        
        # Ensure 'age' is consistently typed if tally_by_age is True
        if tally_by_age and 'age' in df.columns:
            df['age'] = df['age'].astype(str)

        # Select the required columns based on the tally_by_age flag
        group_cols = [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in df.columns else [])

        # Ensure selected columns exist in the DataFrame
        missing_cols = [col for col in group_cols if col not in df.columns]
        if missing_cols:
            raise KeyError(f"The DataFrame '{df_name}' is missing columns: {missing_cols}")

        # Remove duplicate rows based on the selected grouping columns
        subset = df[group_cols].drop_duplicates()
        
        # Add a count column for tallying occurrences
        subset['count'] = 1

        # Aggregate the counts by the grouping columns
        grouped = subset.groupby(group_cols).count().reset_index()

        # Rename the count column to indicate which DataFrame it came from
        grouped.rename(columns={'count': df_name}, inplace=True)

        # Merge the current tally with the combined tally
        if combined_tally.empty:
            combined_tally = grouped
        else:
            combined_tally = pd.merge(combined_tally, grouped, on=group_cols, how='outer')

    # Fill NaN values with 0 (indicating no occurrences in that DataFrame)
    combined_tally.fillna(0, inplace=True)

    # Calculate the total tally across all DataFrame columns
    tally_columns = combined_tally.columns.difference(group_cols)
    combined_tally['total'] = combined_tally[tally_columns].sum(axis=1)

    # Sort the DataFrame by the total count in descending order
    combined_tally.sort_values(by='total', ascending=False, inplace=True)

    # --- New logic to handle multiple score column names ---

    # Concatenate all the DataFrames in the outlier_dfs dictionary
    combined_df = pd.concat(outlier_dfs.values())

    # Check if the DataFrame contains 'robust_z_score' or 'z_score' and create a unified column
    if 'robust_z_score' in combined_df.columns:
        combined_df['outlier_score'] = combined_df['robust_z_score']
    elif 'z_score' in combined_df.columns:
        combined_df['outlier_score'] = combined_df['z_score']
    else:
        raise ValueError("None of the DataFrames contain 'robust_z_score' or 'z_score'.")

    # Group by 'gss_code_ward' (or 'gss_code'), 'year', and 'age', and sum the outlier scores
    group_outliers = combined_df.groupby(
        [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in combined_df.columns else []),
        as_index=False).agg(total_outlier_score=('outlier_score', lambda x: x.abs().sum())) #use abs() so that the magnitude of negative and positive outliers are counted for

    # Merge the total outlier score with the combined_tally DataFrame
    final_df = pd.merge(
        combined_tally, group_outliers, 
        on=[geo_column, 'year'] + (['age'] if tally_by_age and 'age' in group_outliers.columns else []), 
        how='left'
    )

    return final_df


In [ ]:
def find_ahs_outliers_absolute_value(df, z_threshold=3):
    """
    Identifies outliers in the average household size (ahs) dataframe based on z-scores and returns a DataFrame with additional context.
    
    Args:
    - df (pd.DataFrame): DataFrame with household size data. First column should be 'gss_code_ward'.
    - z_threshold (float): Threshold for z-scores to determine outliers. Default is 3.
    
    Returns:
    - pd.DataFrame: DataFrame containing gss_code_ward, year, z_score, outlier value, year_before, year_after,
                    average_for_year (across wards), and average_for_ward (across years).
    """
    
    # Step 1: Calculate z-scores for each year column (excluding 'gss_code_ward')
    z_scores = df.iloc[:, 1:].apply(zscore)

    # Step 2: Identify the outliers (z-scores > z_threshold or z-scores < -z_threshold)
    outliers = z_scores[(z_scores > z_threshold) | (z_scores < -z_threshold)]

    # Step 3: Combine 'gss_code_ward' with outliers to present them in a readable format
    outliers_cleaned = pd.concat([df['gss_code_ward'], outliers], axis=1)

    # Step 4: Drop rows with no outliers
    outliers_cleaned = outliers_cleaned.dropna(how='all', subset=outliers_cleaned.columns[1:])

    # Step 5: Melt the DataFrame to have 'gss_code_ward', 'year', and 'z_score' columns
    outliers_melted = outliers_cleaned.melt(id_vars='gss_code_ward', var_name='year', value_name='z_score')

    # Step 6: Drop NaN values (non-outliers) and sort by z_score in descending order
    outliers_sorted = outliers_melted.dropna().sort_values(by='z_score', ascending=False)

    # Convert the 'year' column to integer for easier manipulation
    outliers_sorted['year'] = outliers_sorted['year'].astype(int)

    # Step 7: Create new columns for the value of AHS for outlier year, previous year, and next year
    outliers_sorted['outlier_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'])].values[0], axis=1)

    outliers_sorted['year_before_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] - 1)].values[0]
        if str(row['year'] - 1) in df.columns else None, axis=1)

    outliers_sorted['year_after_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] + 1)].values[0]
        if str(row['year'] + 1) in df.columns else None, axis=1)

    # Step 8: Calculate the average for that year across all gss_code_wards
    outliers_sorted['average_for_year'] = outliers_sorted.apply(
        lambda row: df[str(row['year'])].mean(), axis=1)

    # Step 9: Calculate the average for that gss_code_ward across all years
    outliers_sorted['average_for_ward'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], df.columns[1:]].mean(axis=1).values[0], axis=1)

    # Step 10: Create the final outlier_sorted_context DataFrame
    outlier_sorted_context = outliers_sorted[['gss_code_ward', 'year', 'z_score', 'outlier_value', 'year_before_value', 'year_after_value', 'average_for_year', 'average_for_ward']]

    return outlier_sorted_context

# Example usage:
#outliers = find_outliers_with_context(average_household_size, z_threshold=3)

# If you want to use a different threshold, e.g., z-score threshold of 2.5:
#outlier_sorted_context = find_outliers_with_context(average_household_size, z_threshold=2.5)

# Display the results
#print(outlier_sorted_context)


In [ ]:
def find_ahs_outliers_pct_change(df, pct_change_threshold=0.3):
        """
        Identifies outliers in the average household size (ahs) dataframe based on percentage change and returns a DataFrame with additional context.
        
        Args:
        - df (pd.DataFrame): DataFrame with household size data. First column should be 'gss_code_ward'.
        - pct_change_threshold (float): Threshold for percentage change to determine outliers. Default is 0.2 (20%).
        
        Returns:
        - pd.DataFrame: DataFrame containing gss_code_ward, year, pct_change, outlier value, year_before, year_after,
                        average_for_year (across wards), and average_for_ward (across years).
        """
        
        # Step 1: Calculate percentage change for each year column (excluding 'gss_code_ward')
        pct_changes = df.iloc[:, 1:].pct_change(axis=1)
        
        # Step 2: Identify the outliers (percentage change > pct_change_threshold or < -pct_change_threshold)
        outliers = pct_changes[(pct_changes > pct_change_threshold) | (pct_changes < -pct_change_threshold)]
  
        # Step 3: Combine 'gss_code_ward' with outliers to present them in a readable format
        outliers_cleaned = pd.concat([df['gss_code_ward'], outliers], axis=1)

        # Step 4: Drop rows with no outliers
        outliers_cleaned = outliers_cleaned.dropna(how='all', subset=outliers_cleaned.columns[1:])
        
        # Step 5: Melt the DataFrame to have 'gss_code_ward', 'year', and 'pct_change' columns
        outliers_melted = outliers_cleaned.melt(id_vars='gss_code_ward', var_name='year', value_name='pct_change')

        # Step 6: Drop NaN values (non-outliers) and sort by pct_change in descending order
        outliers_sorted = outliers_melted.dropna().sort_values(by='pct_change', ascending=False)

        # Convert the 'year' column to integer for easier manipulation
        outliers_sorted['year'] = outliers_sorted['year'].astype(int)

        # Step 7: Create new columns for the value of AHS for outlier year, previous year, and next year
        outliers_sorted['outlier_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'])].values[0], axis=1)

        outliers_sorted['year_before_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] - 1)].values[0]
            if str(row['year'] - 1) in df.columns else None, axis=1)

        outliers_sorted['year_after_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] + 1)].values[0]
            if str(row['year'] + 1) in df.columns else None, axis=1)

        # Step 8: Calculate the average for that year across all gss_code_wards
        outliers_sorted['average_for_year'] = outliers_sorted.apply(
            lambda row: df[str(row['year'])].mean(), axis=1)

        # Step 9: Calculate the average for that gss_code_ward across all years
        outliers_sorted['average_for_ward'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], df.columns[1:]].mean(axis=1).values[0], axis=1)

        # Step 10: Create the final outlier_sorted_context DataFrame
        outlier_sorted_context = outliers_sorted[['gss_code_ward', 'year', 'pct_change', 'outlier_value', 'year_before_value', 'year_after_value', 'average_for_year', 'average_for_ward']]

        return outlier_sorted_context

    # Example usage:
    # outlier_sorted_context = find_ahs_outliers_with_pct_change(average_household_size, pct_change_threshold=0.2)

    # If you want to use a different threshold, e.g., percentage change threshold of 15%:
    # outlier_sorted_context = find_ahs_outliers_with_pct_change(average_household_size, pct_change_threshold=0.15)

    # Display the results
    # print(outlier_sorted_context)

## Process data

---

In [ ]:
#split ward and borough dataframes
combined_10yr_fert_boroughs = combined_10yr_fert[combined_10yr_fert['gss_code_ward'].isna()]
combined_10yr_fert_ward = combined_10yr_fert[~combined_10yr_fert['gss_code_ward'].isna()]

In [ ]:
#bin ages
combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)

In [ ]:
#seperate components into columns
combined_10yr_fert_agebins_component_columns = combined_10yr_fert_agebins.pivot_table(index=['gss_code','gss_code_ward','sex', 'age','year'], columns='component', values='value').reset_index()

In [ ]:
### Create ward and borough names lookup

In [ ]:
name_lookup = combined_10yr_fert[['gss_code', 'la_name', 'gss_code_ward', 'ward_name']]
#remove where all values in the row are the same
name_lookup = name_lookup.drop_duplicates()
#remove nan values
name_lookup = name_lookup.dropna()

In [ ]:
# Identify duplicated ward names and create a unique name by appending the local authority name
name_lookup['ward_name_unique'] = name_lookup.apply(
    lambda row: f"{row['ward_name']} ({row['la_name']})" if row['ward_name'] in name_lookup[name_lookup['ward_name'].duplicated()]['ward_name'].values else row['ward_name'],
    axis=1
)

## Basic Checks
Perform basic checks on the dataset, including checking for missing values, duplicates, and descriptive statistics.

---

In [ ]:
#min and max year
def get_year_range(df):
    return df['year'].max(), df['year'].min()

In [ ]:
#year ranges
print("Complete year range:", get_year_range(combined_10yr_fert))
print("Year range for wards:", get_year_range(combined_10yr_fert_ward))
print("Year range for boroughs:", get_year_range(combined_10yr_fert_boroughs))

##### missing values

In [ ]:
missing_values = combined_10yr_fert.isnull().sum()
print("Missing values per column:\n", missing_values)

#### duplicates

In [ ]:
duplicates = combined_10yr_fert.duplicated().sum()
print("Number of duplicate rows:", duplicates)

##### Descriptive data

##### Description by components

In [ ]:
combined_10yr_fert_agebins_component_columns

#### check for negative values in columns

In [ ]:
# Checking for negative values and extremely high values
negative_values = combined_10yr_fert[combined_10yr_fert['value'] < 0]
print('components with negative values:', negative_values['component'].unique())

#### check age range

In [ ]:
#print true if max age is 90 and min age is 0
print('Is max age is 90 and min age is 0:', (combined_10yr_fert['age'].max() == 90) & (combined_10yr_fert['age'].min() == 0))

## Population Consistency Over Time

---

In [ ]:
#use binned ages this will even out large flunctions between age group where the are likely to be unusally high e.i. 18 year olds moving to university
combined_10yr_fert_agebins_component_columns

In [ ]:
outliers_dict_borough = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

In [ ]:
outliers_dict_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False
    )

In [ ]:
outliers_dict_borough_inf_values = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

In [ ]:
outliers_dict_ward_inf_values = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False)

In [ ]:
births_pct_change_borough_outlier_df = outliers_dict_borough['births_pct_change']
deaths_pct_change_borough_outlier_df = outliers_dict_borough['deaths_pct_change']
netflow_pct_change_borough_outlier_df = outliers_dict_borough['netflow_pct_change']
popn_pct_change_borough_outlier_df = outliers_dict_borough['popn_pct_change']

In [ ]:
births_pct_change_ward_outlier_df = outliers_dict_ward['births_pct_change']
deaths_pct_change_ward_outlier_df = outliers_dict_ward['deaths_pct_change']
netflow_pct_change_ward_outlier_df = outliers_dict_ward['netflow_pct_change']
popn_pct_change_ward_outlier_df = outliers_dict_ward['popn_pct_change']

In [ ]:
births_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['births']
deaths_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['deaths']
netflow_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['netflow']
popn_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['popn']

In [ ]:
births_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['births']
deaths_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['deaths']
netflow_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['netflow']   
popn_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['popn']

## Total Population Outliers
#### Detect total outliers using Z-scores and Robust Z-scores, and perform cross-sectional and temporal comparisons.

#### Using total population (popn) perform cross-sectional comparisons: Examine changes between boroughs and wards totals for a given year.
#### Conduct temporal comparisons: Measure percentage changes between year total for both boroughs and wards

#### Total population per geographical boundary
#### Total population per year
#### i.e.
#### groupby('gss_code')['popn']
#### groupby('year')['popn']
---

In [ ]:
combined_10yr_fert_agebins_component_columns

In [ ]:
total_pop_temporal_outliers_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='temporal'
    )

In [ ]:
total_pop_temporal_outliers_ward_df  = total_pop_temporal_outliers_ward['popn_pct_change']

In [ ]:
total_pop_cross_sectional_outliers_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='cross-sectional'
    )

In [ ]:
total_pop_cross_sectional_outliers_ward_df = total_pop_cross_sectional_outliers_ward['popn_pct_change']

## Gender Outliers
Investigate gender outliers, focusing on abnormal gender ratios and adjusting thresholds as needed.

---

In [ ]:
#single year age 
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

In [ ]:
#binned age
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert_agebins, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

In [ ]:
gender_outlier_dictionary
births_gender_outliers_df = gender_outlier_dictionary['births']
deaths_gender_outliers_df = gender_outlier_dictionary['deaths']
netflow_gender_outliers_df = gender_outlier_dictionary['netflow']
popn_gender_outliers_df = gender_outlier_dictionary['popn']

In [ ]:
births_gender_outliers_df

In [ ]:
gender_outlier_dictionary

## Key Visualisations
Visualise important trends in the dataset, including distribution by components, age groups, yearly totals, and population pyramids.

---


#### yearly totals by component visulation

In [ ]:
yearly_totals = combined_10yr_fert.groupby(['year','component'])['value'].sum().reset_index()
print("Yearly population totals by gss_code and year:\n", yearly_totals)

In [ ]:
# Create the line plot with markers
fig = px.line(
    yearly_totals, 
    x='year', 
    y='value', 
    color='component', 
    markers=True,
    title="Yearly Totals by Component Over the Years",
    labels={'value': 'Total Value', 'year': 'Year'},
    color_discrete_sequence=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Four distinct colors
)

# Update layout for refined appearance
fig.update_layout(
    title={
        'text': "Yearly Totals by Component Over the Years",
        'x': 0.5,  # Center the title
        'xanchor': 'center',  # Anchor the title to the center
        'yanchor': 'top'
    },
    title_font=dict(size=20, color='#333333', family='Arial'),
    xaxis_title='Year',
    yaxis_title='Total Value',
    legend_title='<b>Component</b><br>(Please click components<br> to toggle visibility)',
    width=900,
    height=600,
    font=dict(family="Arial", size=14, color="#333333"),
    plot_bgcolor='#f7f7f7',
    paper_bgcolor='#ffffff',
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        x=0.5,
        xanchor="center",
        font=dict(size=12)
    ),
    margin=dict(l=60, r=60, t=60, b=60)  # Set margins to adjust spacing around the plot
)

# Show the figure
fig.show()

# Save the figure as an HTML file for embedding
pio.write_html(fig, 'interactive_line_plot.html', include_plotlyjs='cdn')



In [ ]:

# Create a bar graph using Plotly Express
fig = px.line(
    yearly_totals, 
    x='year', 
    y='value', 
    color='component', 
    markers=True,
    title="Yearly Totals by Component Over the Years",
    labels={'value': 'Total Value', 'year': 'Year'},
    
)

# Update layout for better appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Value',
    legend_title='<b>Component</b><br>(Please click components<br> to toggle visibility)',
    width=900,
    height=600
)

# Show the figure
fig.show()
# Save the figure as an HTML file for embedding
pio.write_html(fig, 'interactive_line_plot.html', include_plotlyjs='cdn')

#### population pyramid

In [ ]:
#for unit age
combined_10yr_fert_popn = combined_10yr_fert[combined_10yr_fert['component'] == 'popn']
population_pyramids_unit_age = combined_10yr_fert_popn.copy()
population_pyramids_unit_age = population_pyramids_unit_age[~population_pyramids_unit_age['gss_code_ward'].isna()]

In [ ]:
population_pyramids_unit_age

population_pyramids_unit_age.info(memory_usage='deep')

In [ ]:
#sum count for la_name for each year and gender
population_pyramids_unit_age['LA_total'] = population_pyramids_unit_age.groupby(['la_name', 'year', 'sex', 'age'])['value'].transform('sum')


In [ ]:
# Initialize the Dash app with a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define layout with CSS styling
app.layout = html.Div([
    html.H2("Population Pyramid Analysis", className="text-center mt-4", style={'color': '#333333', 'font-size': '24px'}),
    
    # Dropdown Container
    html.Div([
        dbc.Row([
            dbc.Col([
                html.Label("Select Borough", style={'font-size': '16px', 'margin-bottom': '5px', 'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='location-dropdown',
                    options=[{'label': loc, 'value': loc} for loc in population_pyramids_unit_age['la_name'].unique()],
                    value=population_pyramids_unit_age['la_name'].unique()[0],
                    style={'width': '100%', 'font-size': '16px'}
                )
            ], width=6),
            
            dbc.Col([
                html.Label("Select Ward", style={'font-size': '16px', 'margin-bottom': '5px', 'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='ward-dropdown',
                    style={'width': '100%', 'font-size': '16px'}
                )
            ], width=6)
        ], className="row justify-content-center"),
    ], className="container", style={'padding': '20px'}),
    
    # Graph
    dcc.Graph(id='population-pyramid'),
    
], style={'padding': '20px', 'font-family': 'Arial', 'background-color': '#f7f7f7'})

# Update ward dropdown based on selected borough
@app.callback(
    Output('ward-dropdown', 'options'),
    Output('ward-dropdown', 'value'),
    Input('location-dropdown', 'value')
)
def update_ward_dropdown(selected_location):
    filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location]
    ward_options = [{'label': ward, 'value': ward} for ward in filtered_data['ward_name'].unique()]
    ward_options.append({'label': 'All Wards', 'value': 'All Wards'})  # Add "All Wards" as the last option
    return ward_options, ward_options[0]['value']  # Set the first actual ward as default

# Generate the population pyramid chart
@app.callback(
    Output('population-pyramid', 'figure'),
    Input('location-dropdown', 'value'),
    Input('ward-dropdown', 'value')
)
def update_pyramid(selected_location, selected_ward):
    # If a specific location is selected with "All Wards", use LA_total for that location
    if selected_ward == 'All Wards':
        filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location].copy()
        filtered_data['value'] = filtered_data['LA_total']  # Use LA_total for overall population
        title = f'Population Pyramid for {selected_location} - All Wards'
    
    # For a specific ward
    else:
        filtered_data = population_pyramids_unit_age[
            (population_pyramids_unit_age['la_name'] == selected_location) & 
            (population_pyramids_unit_age['ward_name'] == selected_ward)
        ]
        title = f'Population Pyramid for {selected_location} - {selected_ward}'

    # Prepare data for the pyramid
    filtered_data['value'] = filtered_data.apply(lambda row: -row['value'] if row['sex'] == 'female' else row['value'], axis=1)

    # Calculate range_x based on selection type
    if selected_ward == 'All Wards':
        max_value = abs(filtered_data['value'].max() * len(filtered_data['ward_name'].unique()))
        range_x = [-max_value * 1.2, max_value * 1.2] 
    else:
        max_value = abs(filtered_data['value'].max())
        range_x = [-max_value * 1.2, max_value * 1.2]

    fig = px.bar(
        filtered_data,
        x='value',
        y='age',
        color='sex',
        animation_frame='year',
        orientation='h',
        title=title,
        labels={'value': 'Population', 'age': 'Age'},
        color_discrete_map={'male': '#1f77b4', 'female': '#ff7f0e'},
        height=600,
        range_x=range_x,
        hover_data={'ward_name': True}
    )

    # Custom layout for enhanced appearance
    fig.update_layout(
        barmode='relative',
        title={'text': title, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        title_font=dict(size=20, color='#333333', family='Arial'),
        xaxis_title='Population',
        yaxis_title='Age',
        font=dict(family="Arial", size=14, color="#333333"),
        xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
        yaxis=dict(showgrid=True, gridcolor='lightgrey'),
        plot_bgcolor='#f7f7f7',
        paper_bgcolor='#ffffff',
        showlegend=True,
        legend=dict(title="Gender", orientation="h", y=1.1, x=0.5, xanchor="center")
    )

    # Set a default animation speed for transitions
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500  # Default animation frame duration
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 250  # Transition duration

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=1223)


In [ ]:
#add ward and borough names
combined_10yr_fert_agebins_component_columns_with_names = pd.merge(combined_10yr_fert_agebins_component_columns, name_lookup, left_on='gss_code_ward', right_on='gss_code_ward', how='left') 

In [ ]:
data = combined_10yr_fert_agebins_component_columns_with_names.copy()

In [ ]:
# Initialize the Dash app with a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App layout
app.layout = html.Div([
    html.H1("Ward Demographic Dashboard", style={'textAlign': 'center', 'color': '#333333', 'margin-top': '20px'}),
    
    # Dropdown for selecting ward
    html.Div([
        dbc.Row([
            dbc.Col([
                html.Label("Select Ward", style={'font-size': '16px', 'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='ward-dropdown',
                    options=[{'label': ward, 'value': ward} for ward in data['ward_name_unique'].unique()],
                    value=data['ward_name_unique'].unique()[0],
                    style={'font-size': '16px', 'margin-bottom': '20px'}
                )
            ], width=6),
            dbc.Col([
                html.Label("Select Component", style={'font-size': '16px', 'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='component-dropdown',
                    options=[
                        {'label': 'Births', 'value': 'births'},
                        {'label': 'Deaths', 'value': 'deaths'},
                        {'label': 'Netflow', 'value': 'netflow'},
                        {'label': 'Population', 'value': 'popn'},
                        {'label': 'Births % Change', 'value': 'births_pct_change'},
                        {'label': 'Deaths % Change', 'value': 'deaths_pct_change'},
                        {'label': 'Netflow % Change', 'value': 'netflow_pct_change'},
                        {'label': 'Population % Change', 'value': 'popn_pct_change'}
                    ],
                    value='births',
                    style={'font-size': '16px'}
                )
            ], width=6)
        ], style={'margin-bottom': '20px'})
    ], style={'padding': '20px'}),
    
    # Dropdown for selecting age group
    html.Div([
        html.Label("Select Age Group", style={'font-size': '16px', 'font-weight': 'bold'}),
        dcc.Dropdown(id='age-dropdown', style={'font-size': '16px'})
    ], style={'padding': '10px'}),
    
    # Radio buttons for selecting plot type
    html.Div([
        html.Label("Select Plot Type", style={'font-size': '16px', 'font-weight': 'bold'}),
        dcc.RadioItems(
            id='plot-type',
            options=[
                {'label': 'Line Plot', 'value': 'line'},
                {'label': 'Box Plot', 'value': 'box'}
            ],
            value='line',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        ),
    ], style={'padding': '10px'}),
    
    # Graph placeholder
    dcc.Graph(id='plot', style={'margin-top': '20px'})
])

# Callback to update age dropdown options based on selected component
@app.callback(
    Output('age-dropdown', 'options'),
    Output('age-dropdown', 'value'),
    Input('component-dropdown', 'value')
)
def update_age_dropdown(selected_component):
    if selected_component == 'births':
        # Allow only age 0 when "Births" is selected
        options = [{'label': 'Age 0', 'value': '0-18'}]
        value = '0-18'
    else:
        # Show all available age options and "All Ages"
        options = [{'label': age, 'value': age} for age in data['age'].unique()] + [{'label': 'All Ages', 'value': 'all'}]
        value = 'all'
    return options, value

# Update graph based on dropdowns and selected plot type
@app.callback(
    Output('plot', 'figure'),
    [Input('ward-dropdown', 'value'),
     Input('component-dropdown', 'value'),
     Input('age-dropdown', 'value'),
     Input('plot-type', 'value')]
)
def update_plot(selected_ward, selected_component, selected_age, selected_plot_type):
    # Filter data by selected ward
    filtered_data = data[data['ward_name_unique'] == selected_ward]
    
    # Filter by age group if not "all"
    if selected_age != 'all':
        filtered_data = filtered_data[filtered_data['age'] == selected_age]
    
    if selected_plot_type == 'line':
        if selected_age == 'all':
            # Aggregate data by year and sex for "All Ages"
            aggregated_data = filtered_data.groupby(['year', 'sex'])[selected_component].sum().reset_index()
            fig = px.line(
                aggregated_data,
                x='year',
                y=selected_component,
                color='sex',
                title=f"{selected_component.capitalize()} Trend Over Time in {selected_ward} (All Ages)"
            )
        else:
            # Create a line plot without aggregation
            fig = px.line(
                filtered_data,
                x='year',
                y=selected_component,
                color='sex',
                title=f"{selected_component.capitalize()} Trend Over Time in {selected_ward}"
            )
        fig.update_layout(
            xaxis_title="Year",
            yaxis_title=selected_component,
            plot_bgcolor="white"
        )
    
    elif selected_plot_type == 'box':
        # Create a box plot
        fig = px.box(
            filtered_data,
            x='year',
            y=selected_component,
            color='sex',
            title=f"{selected_component.capitalize()} Distribution by Gender in {selected_ward} (Box Plot)"
        )
        fig.update_layout(
            xaxis_title="Year",
            yaxis_title=selected_component,
            plot_bgcolor="white"
        )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=1333)




In [ ]:
# Define the dropdown options
dropdown_options = [
    {'label': 'Births', 'value': 'births'},
    {'label': 'Deaths', 'value': 'deaths'},
    {'label': 'Netflow', 'value': 'netflow'},
    {'label': 'Population', 'value': 'popn'},
    {'label': 'Births % Change', 'value': 'births_pct_change'},
    {'label': 'Deaths % Change', 'value': 'deaths_pct_change'},
    {'label': 'Netflow % Change', 'value': 'netflow_pct_change'},
    {'label': 'Population % Change', 'value': 'popn_pct_change'}
]

# Initial plot (default column)
fig = px.histogram(data, x='deaths', title='Interactive Histogram with Dropdown')

# Update layout to add dropdown
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [{'x': [data[option['value']]]},  # Update the x-axis data
                             {'xaxis.title.text': option['label']}],  # Update x-axis label
                    'label': option['label'],
                    'method': 'update'
                }
                for option in dropdown_options
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.5,
            'y': 1.15
        }
    ]
)

# Show the figure
fig.show()


In [ ]:
combined_10yr_fert_agebins_component_columns_with_names

In [ ]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

df = combined_10yr_fert_agebins_component_columns_with_names.copy()

# Preprocess the data: Ensure the relevant columns are numeric
columns_to_visualize = ['births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change']
df[columns_to_visualize] = df[columns_to_visualize].apply(pd.to_numeric, errors='coerce')

# Initialize the Dash app with a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Dropdown options for columns
column_options = [{'label': col, 'value': col} for col in columns_to_visualize]

# Colors for 'male' and 'female'
gender_colors = {
    'male': '#1f77b4',  # blue
    'female': '#ff7f0e'  # orange
}

# Create the swarm plot function
def create_swarm_plot(selected_column):
    min_year = df['year'].min()
    max_year = df['year'].max()

    # Calculate range_y based on selection type
    max_value = abs(df[selected_column].max())
    range_y = [-max_value * 1.2, max_value * 1.2]
    
    # Create the figure with animation
    fig = px.strip(
        df,  # Use the full DataFrame for animation across years
        x='age',  # Age on the x-axis
        y=selected_column,  # Selected column on the y-axis
        color='sex',  # Color by sex
        color_discrete_map=gender_colors,
        animation_frame='year',  # Animation by year
        title=f"{selected_column.capitalize()} by Age Over Time",
        hover_data=['ward_name', 'la_name', 'sex'],  # Hover data
        category_orders={'age': sorted(df['age'].unique())},  # Sort ages on x-axis
        range_y=range_y
    )
    
    # Update layout for readability
    fig.update_layout(
        xaxis_title="Age Group",
        yaxis_title=selected_column.capitalize(),
        template="plotly_dark",  # Change to dark theme
        height=600,
        font=dict(family="Arial, sans-serif", size=14),
        margin=dict(l=50, r=50, t=50, b=50),
        showlegend=True
    )
    
    return fig

# Define the layout of the app
app.layout = html.Div([
    # Header section
    html.Div([
        html.H1("Interactive Swarm Plot by Age", style={'textAlign': 'center', 'color': '#2C3E50'}),
        html.P("Select the column to visualize the data animated by year", 
               style={'textAlign': 'center', 'fontSize': '18px', 'color': '#7F8C8D'}),
    ], style={'padding': '20px', 'backgroundColor': '#ECF0F1'}),
    
    # Dropdown for column selection
    dbc.Row([
        dbc.Col([
            dbc.Label("Select Metric", style={'fontSize': '16px'}),
            dcc.Dropdown(
                id='column-dropdown',
                options=column_options,
                value=columns_to_visualize[0],  # Default value
                style={'width': '100%'}
            ),
        ], width=6)
    ], style={'padding': '20px'}),
    
    # Graph section
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='swarm-plot'),
        ], width=12)
    ], style={'padding': '20px'}),
])

# Callback to update the plot based on user input
@app.callback(
    Output('swarm-plot', 'figure'),
    [Input('column-dropdown', 'value')]
)
def update_swarm_plot(selected_column):
    return create_swarm_plot(selected_column)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=2277)



## Collate Outliers
Summarize and compile all detected outliers for further analysis.

---

In [ ]:
# Filter the global variables to find dataframes with 'outlier' in their name
outlier_dfs = {name: df for name, df in globals().items() if isinstance(df, pd.DataFrame) and 'outlier' in name.lower()}

# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")

In [ ]:
# Only ward data datatframes
# Filter the global variables to find dataframes with 'outlier' in their name and exclude ones with 'borough'
outlier_dfs_wards = {name: df for name, df in globals().items() 
               if isinstance(df, pd.DataFrame) 
               and 'outlier' in name.lower() 
               and 'borough' not in name.lower()}
# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs_wards.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")

In [ ]:
# View age column in dfs
age_snippets = {df_name: df['age'].iloc[0] for df_name, df in outlier_dfs_wards.items()}
age_snippets

In [ ]:
# Initialise an empty list to store relevant rows from all DataFrames
all_rows = []

# Loop through all outlier dataframes
for name, df in outlier_dfs_wards.items():
    # Check if 'gss_code' and 'year' columns exist
    if 'gss_code_ward' in df.columns and 'year' in df.columns:
        # Select the relevant columns: 'gss_code', 'year' and 'age' (if it exists)
        cols = ['gss_code_ward', 'year']
        if 'age' in df.columns:
            cols.append('age')

        # Append the relevant data from the current DataFrame to the list
        all_rows.append(df[cols])

# Concatenate all collected data into one DataFrame
if all_rows:
    combined_df = pd.concat(all_rows)

    # Group by 'gss_code', 'year', and 'age' (where applicable) and count occurrences
    tally_df = combined_df.groupby(cols).size().reset_index(name='count')

    # Sort the result by count in descending order
    tally_df = tally_df.sort_values(by='count', ascending=False)

    # Display the top rows of the tally DataFrame
    print(tally_df)
else:
    print("No relevant data found.")

In [ ]:
outlier_dfs_wards

In [ ]:
df_tally = tally_outliers(outlier_dfs_wards, tally_by_age=True)


In [ ]:
df_tally

In [ ]:
df_tally

In [ ]:
# Concatenate all the DataFrames together
combined_df = pd.concat(outlier_dfs_wards)

# Group by 'gss_code_ward', 'year', and 'age', and sum the outlier scores
grouped_df = combined_df.groupby(['gss_code_ward', 'year', 'age'], as_index=False).agg(
    total_outlier_score=('robust_z_score', 'sum')
)

In [ ]:
grouped_df

In [ ]:
grouped_df
grouped_df.sort_values(by='total_outlier_score', ascending=False)

In [ ]:
# potential alterations:
# using single year of age vs binned ages 
# add weighting to final tally
# remove certain age groups like 90+ from outlier detection
# household data
# hmtl report generation


## Average Household Size Outliers
This section will use the function to produce a dataframe with outliers regrading the average household size.
It also contains some contextual results so comparison can be more easily made, such as the year before and after, so that teh change can be seen. As well as the average household size for that word as well as that year across all wards.

In [ ]:
#read in rds file
average_household_size = pyreadr.read_r(r"C:\Users\user\Documents\population_data\2022_Identified_Capacity_15yr_high_fert\2022_Identified_Capacity_15yr_high_fert\ahs.rds")
average_household_size = average_household_size[None]

In [ ]:
#remove nan columns
average_household_size = average_household_size.dropna(axis=1, how='all')

In [ ]:
average_household_size

In [ ]:
# Example usage:
anomalous_average_household_size_absolute = find_ahs_outliers_absolute_value(average_household_size, z_threshold=3)



In [ ]:
anomalous_average_household_size_absolute

In [ ]:
#unique values in gss_code_ward for outlier_sorted_context
print("outlier wards:", anomalous_average_household_size_absolute['gss_code_ward'].unique())

In [ ]:
anomalous_average_household_size_pct_change = find_ahs_outliers_pct_change(average_household_size, pct_change_threshold=0.10
                                                               )

In [ ]:
anomalous_average_household_size_pct_change 

## Produce HMTL report

##### Table formatting got html

In [ ]:
# Define the DataFrames, shorten  
tally_df_20 = df_tally.head(20)  # DataFrame containing outliers
anomalous_average_household_size_absolute_df = anomalous_average_household_size_absolute.head(20)  # DataFrame with summary statistics
anomalous_average_household_size_pct_change_df = anomalous_average_household_size_pct_change  # DataFrame with outliers    

In [ ]:
name_lookup = combined_10yr_fert[['gss_code', 'la_name', 'gss_code_ward', 'ward_name']]
#remove where all values in the row are the same
name_lookup = name_lookup.drop_duplicates()
#remove nan values
name_lookup = name_lookup.dropna()

In [ ]:
# Identify duplicated ward names and create a unique name by appending the local authority name
name_lookup['ward_name_unique'] = name_lookup.apply(
    lambda row: f"{row['ward_name']} ({row['la_name']})" if row['ward_name'] in name_lookup[name_lookup['ward_name'].duplicated()]['ward_name'].values else row['ward_name'],
    axis=1
)


In [ ]:
#duplicate ward_name
name_lookup

In [ ]:
#merge dataframes with name_lookup
tally_df_20 = pd.merge(tally_df_20, name_lookup, left_on='gss_code_ward', right_on='gss_code_ward', how='left') 
anomalous_average_household_size_absolute_df = pd.merge(anomalous_average_household_size_absolute_df, name_lookup, left_on='gss_code_ward', right_on='gss_code_ward', how='left')   
anomalous_average_household_size_pct_change_df = pd.merge(anomalous_average_household_size_pct_change_df, name_lookup, left_on='gss_code_ward', right_on='gss_code_ward', how='left')   

In [ ]:
#rename column heading
#tally_df = tally_df.rename(columns={'gss_code': 'Borough Code', 'ward_name': 'Ward Name', 'gss_code_ward': 'Ward Code', 'ward_name_unique': 'Unique Ward Name', 'la_name': 'Borough Name', 'year': 'Year', 'age': 'Age', 'births_pct_change_ward_outlier_df': 'Births % Change Outlier', 'deaths_pct_change_ward_outlier_df': 'Deaths % Change Outlier', 'netflow_pct_change_ward_outlier_df': 'Netflow % Change Outlier', 'popn_pct_change_ward_outlier_df': 'Population % Change Outlier', 'births_pct_change_ward_inf_outlier_df': 'Births % Change Outlier (Inf Values)', 'deaths_pct_change_ward_inf_outlier_df': 'Deaths % Change Outlier (Inf Values)', 'netflow_pct_change_ward_inf_outlier_df': 'Netflow % Change Outlier (Inf Values)', 'popn_pct_change_ward_inf_outlier_df': 'Population % Change Outlier (Inf Values)', 'total_pop_temporal_outliers_ward_df': 'Total Population Temporal Outlier', 'total_pop_cross_sectional_outliers_ward_df': 'Total Population Cross-Sectional Outlier'})
#drop column ward_name
tally_df_20= tally_df_20.drop(columns=['ward_name'])
anomalous_average_household_size_absolute_df = anomalous_average_household_size_absolute_df.drop(columns=['ward_name'])
anomalous_average_household_size_pct_change_df = anomalous_average_household_size_pct_change_df.drop(columns=['ward_name'])
#rename ward to unique 
tally_df_20 = tally_df_20.rename(columns={'gss_code': 'Borough Code', 'gss_code_ward': 'Ward Code', 'ward_name_unique': 'Ward Name', 'la_name': 'Borough Name', 'year': 'Year', 'age': 'Age', 'total': 'Total Occurrence', 'total_outlier_score': 'Total Outlier Score'})
anomalous_average_household_size_absolute_df = anomalous_average_household_size_absolute_df.rename(columns={'gss_code': 'Borough Code', 'gss_code_ward': 'Ward Code', 'ward_name_unique': 'Unique Ward Name', 'la_name': 'Borough Name', 'year': 'Year', 'z_score': 'Z-Score', 'outlier_value': 'Outlier Value', 'year_before_value': 'Year Before Value', 'year_after_value': 'Year After Value', 'average_for_year': 'Average for Year', 'average_for_ward': 'Average for Ward'})
anomalous_average_household_size_pct_change_df = anomalous_average_household_size_pct_change_df.rename(columns={'gss_code': 'Borough Code', 'gss_code_ward': 'Ward Code', 'ward_name_unique': 'Unique Ward Name', 'la_name': 'Borough Name', 'year': 'Year', 'outlier_value': 'Outlier Value', 'year_before_value': 'Year Before Value', 'year_after_value': 'Year After Value', 'average_for_year': 'Average for Year', 'average_for_ward': 'Average for Ward'})

In [ ]:
anomalous_average_household_size_absolute_df.columns.to_list()

In [ ]:
#order columns
tally_df_20 = tally_df_20[['Borough Name','Borough Code','Ward Name','Ward Code',
 'Year',
 'Age',
 'births_pct_change_ward_outlier_df',
 'deaths_pct_change_ward_outlier_df',
 'netflow_pct_change_ward_outlier_df',
 'popn_pct_change_ward_outlier_df',
 'births_pct_change_ward_inf_outlier_df',
 'deaths_pct_change_ward_inf_outlier_df',
 'netflow_pct_change_ward_inf_outlier_df',
 'popn_pct_change_ward_inf_outlier_df',
 'total_pop_temporal_outliers_ward_df',
 'total_pop_cross_sectional_outliers_ward_df',
 'births_gender_outliers_df',
 'deaths_gender_outliers_df',
 'netflow_gender_outliers_df',
 'popn_gender_outliers_df',
 'Total Occurrence',
 'Total Outlier Score',]]

anomalous_average_household_size_absolute_df = anomalous_average_household_size_absolute_df[['Borough Name','Borough Code', 'Unique Ward Name','Ward Code',
 'Year',
 'Z-Score',
 'Outlier Value',
 'Year Before Value',
 'Year After Value',
 'Average for Year',
 'Average for Ward',
]]

anomalous_average_household_size_pct_change_df = anomalous_average_household_size_pct_change_df[['Borough Name',
    'Borough Code',
    'Unique Ward Name',
    'Ward Code',
    'Year',
    'pct_change',
    'Outlier Value',
    'Year Before Value',
    'Year After Value',
    'Average for Year',
    'Average for Ward']]

#### HTML layout

In [ ]:
# Get the directory where the notebook file is located
script_dir = os.path.dirname(os.path.abspath('QA_population_projection.ipynb'))
# Set the working directory to 'Projections_QA' so the hmtl is saved in the correct location
os.chdir(script_dir)

In [ ]:
# Convert DataFrames to HTML
tally_df_html = tally_df_20.to_html()
average_household_size_absolute_html = anomalous_average_household_size_absolute_df.to_html()
anomalous_average_household_size_pct_change_df_html = anomalous_average_household_size_pct_change_df.to_html()  # Added parentheses

# HTML template for the report with doubled braces in CSS to prevent KeyError
html_content = """
<html>
<head>
    <title>Population Projections Report</title>
    <link rel="icon" href="https://resource.esriuk.com/wp-content/uploads/2017/06/GLA-Logo-Resized.png" type="image/png">  <!-- Link to the favicon -->
    <style>
        /* General Styling */
        body {{
            font-family: Arial, sans-serif;
            color: #333;
            background-color: #f4f4f9;
            margin: 0;
            padding: 0;
        }}
        .container {{
            width: 80%;
            margin: auto;
            background-color: #fff;
            padding: 20px;
            box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.1);
            border-radius: 8px; /* Rounded corners */
        }}
        
        /* Logo Styling */
        .logo {{
            display: block;
            margin: 0 auto;
            width: 250px; /* Increased size */
        }}
        
        /* Header Styling */
        h1, h2 {{
            text-align: center;
            color: #2c3e50;
        }}
        
        h1 {{
            font-size: 2.8em; /* Increased font size */
            border-bottom: 2px solid #2c3e50;
            padding-bottom: 10px;
            margin-bottom: 20px;
        }}
        
        h2 {{
            font-size: 2em; /* Increased font size */
            margin-top: 30px;
        }}
        
        /* Table Container Styling */
        .table-container {{
            max-height: 400px; /* Adjust the height as needed */
            overflow-y: auto;  /* Enables vertical scrolling */
            overflow-x: auto;  /* Enables horizontal scrolling */
            margin: 20px 0;    /* Space above and below the table */
            border: 1px solid #ddd; /* Optional border around the table container */
            padding: 5px; /* Optional padding inside the container */
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-size: 1em;
        }}
        
        table, th, td {{
            border: 1px solid #ddd;
            padding: 12px; /* Increased padding for better readability */
        }}
        
        th {{
            background-color: #2c3e50;
            color: #fff;
            text-align: left;
        }}
        
        tr:nth-child(even) {{
            background-color: #f2f2f2;
        }}
        
        /* Image and Plot Styling */
        .image-container {{
            text-align: center;
            margin-top: 20px;
        }}
        
        .image-container img, .plot-container iframe {{
            max-width: 100%;
            height: auto;
        }}

        /* Footer Styling */
        .footer {{
            text-align: center;
            margin-top: 40px;
            font-size: 1em; /* Increased font size for better readability */
            color: #555;
            padding: 10px 0;
            border-top: 1px solid #ddd; /* Added border to separate footer */
            background-color: #f9f9f9; /* Light background for footer */
        }}
    </style>
</head>
<body>
    <div class="container">
        <!-- Company Logo -->
        <img src="https://resource.esriuk.com/wp-content/uploads/2017/06/GLA-Logo-Resized.png" alt="Company Logo" class="logo">

        <!-- Report Title -->
        <h1>Population Projections Outlier Report</h1>

        <!-- Tallied Dataframe -->
        <h2>Tallied Outlier Table</h2>
        <div class="table-container">
            {tallied_table}
        </div>

        <!-- Average Household Size Absolute Table -->
        <h2>Average Household Size Absolute Table</h2>
        <div class="table-container">
            {average_household_size_absolute}
        </div>

        <!-- Average Household Size Percentage Change Table -->
        <h2>Average Household Size Percentage Change Table</h2>
        <div class="table-container">
            {anomalous_average_household_size_pct}
        </div>

        <!-- Population Pyramid Visualisation -->
        <h2>Interactive Population Pyramid App</h2>
        <div>
            <iframe src="http://localhost:1223" width="100%" height="800" frameborder="0"></iframe>
        </div>

        <!-- Interactive Plot -->
        <h2>Population project by component</h2>
        <div>
            <iframe src="interactive_line_plot.html" width="100%" height="600" frameborder="0"></iframe>
        </div>

        <!-- Footer Section -->
        <div class="footer">
            <p>This is an automated report produced by the Greater London Authority (GLA). The script provided is merely a suggestion for areas to investigate further regarding outliers and potential errors in the population projections. It is not guaranteed to capture every error that may exist within the dataset.</p>
            <p>If you require further information, please email <a href="mailto:Sebastian.Heslin-Rees@london.gov.uk">Sebastian.Heslin-Rees@london.gov.uk</a>.</p>
        </div>
    </div>
</body>
</html>
"""

# Format the HTML content with DataFrame HTML strings
html_report = html_content.format(
    tallied_table=tally_df_html,
    average_household_size_absolute=average_household_size_absolute_html,
    anomalous_average_household_size_pct=anomalous_average_household_size_pct_change_df_html,
)

# Save the report to an HTML file
with open("population_projections_report.html", "w") as f:
    f.write(html_report)

print("HTML report generated successfully!")




In [ ]:
#describe table for all numeric columns
component_columns_describe_value_columns_describe = combined_10yr_fert_agebins_component_columns[['births', 'deaths', 'netflow', 'popn']].describe()
#remove inf and nan values from pct change columns
combined_10yr_fert_agebins_component_columns_no_nans = combined_10yr_fert_agebins_component_columns.replace([np.inf, -np.inf], np.nan).dropna()
#describe percentage change columns
pct_change_columns_describe = combined_10yr_fert_agebins_component_columns_no_nans[['births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change']].describe()
# Concatenate horizontally (side by side)
combined_describe = pd.concat([component_columns_describe_value_columns_describe, pct_change_columns_describe], axis=1)

In [ ]:
combined_describe_html = combined_describe.to_html()

In [ ]:
missing_values = combined_10yr_fert.isnull().sum().to_frame()
missing_values_df = missing_values.rename(columns={0: "missing_values"})
missing_values_df = missing_values_df.to_html()



In [ ]:
# Perform checks and convert results to HTML-safe strings
year_range_fert = get_year_range(combined_10yr_fert)
year_range_ward = get_year_range(combined_10yr_fert_ward)
year_range_boroughs = get_year_range(combined_10yr_fert_boroughs)

missing_values = combined_10yr_fert.isnull().sum().to_frame().to_html()  # Convert to HTML format
duplicates = combined_10yr_fert.duplicated().sum()

# Checking for negative values and extremely high values
negative_values = combined_10yr_fert[combined_10yr_fert['value'] < 0]
negative_components = negative_values['component'].unique()
negative_component_str = negative_components[0]

# Check for the max and min age condition
age_check = (combined_10yr_fert['age'].max() == 90) & (combined_10yr_fert['age'].min() == 0)
max_age = combined_10yr_fert['age'].max()
min_age = combined_10yr_fert['age'].min()

In [ ]:
# Prepare all checks as strings to inject into HTML
check_results = f"""
<h2>Basic sense check</h2>
<h4>Year ranges</h4>
<p>Complete year range: {year_range_fert}</p>
<p>Year range for wards: {year_range_ward}</p>
<p>Year range for boroughs: {year_range_boroughs}</p>

<h4>Missing Values</h4>
<p>Missing values per column:</p>
{missing_values_df}

<h4>Duplicates</h4>
<p>Number of duplicate rows: {duplicates}</p>

<h4>Negative Values</h4>
<p>Components columns that contain negative values: {negative_component_str} </p>


<h4>Age Range Check</h4>
<p>Max age: {max_age}</p>
<p>Min age: {min_age}</p>

<h4>Descriptive Statistics</h4>
<div class="table-container">
    {combined_describe_html}
</div>
"""

# HTML template for the report with placeholders for the checks
html_content = f"""
<html>
<head>
    <title>Population Projections Report</title>
    <link rel="icon" href="https://resource.esriuk.com/wp-content/uploads/2017/06/GLA-Logo-Resized.png" type="image/png">  <!-- Link to the favicon -->
    <style>
        /* General Styling */
        body {{
            font-family: Arial, sans-serif;
            color: #333;
            background-color: #f4f4f9;
            margin: 0;
            padding: 0;
        }}
        .container {{
            width: 80%;
            margin: auto;
            background-color: #fff;
            padding: 20px;
            box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.1);
            border-radius: 8px; /* Rounded corners */
        }}
        
        /* Logo Styling */
        .logo {{
            display: block;
            margin: 0 auto;
            width: 250px; /* Increased size */
        }}
        
        /* Header Styling */
        h1, h2 {{
            text-align: center;
            color: #2c3e50;
        }}
        
        h1 {{
            font-size: 2.8em; /* Increased font size */
            border-bottom: 2px solid #2c3e50;
            padding-bottom: 10px;
            margin-bottom: 20px;
        }}
        
        h2 {{
            font-size: 2em; /* Increased font size */
            margin-top: 30px;
        }}
        
        /* Table Container Styling */
        .table-container {{
            max-height: 400px; /* Adjust the height as needed */
            overflow-y: auto;  /* Enables vertical scrolling */
            overflow-x: auto;  /* Enables horizontal scrolling */
            margin: 20px 0;    /* Space above and below the table */
            border: 1px solid #ddd; /* Optional border around the table container */
            padding: 5px; /* Optional padding inside the container */
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-size: 1em;
        }}
        
        table, th, td {{
            border: 1px solid #ddd;
            padding: 12px; /* Increased padding for better readability */
        }}
        
        th {{
            background-color: #2c3e50;
            color: #fff;
            text-align: left;
        }}
        
        tr:nth-child(even) {{
            background-color: #f2f2f2;
        }}
        
        /* Image and Plot Styling */
        .image-container {{
            text-align: center;
            margin-top: 20px;
        }}
        
        .image-container img, .plot-container iframe {{
            max-width: 100%;
            height: auto;
        }}

        /* Footer Styling */
        .footer {{
            text-align: center;
            margin-top: 40px;
            font-size: 1em; /* Increased font size for better readability */
            color: #555;
            padding: 10px 0;
            border-top: 1px solid #ddd; /* Added border to separate footer */
            background-color: #f9f9f9; /* Light background for footer */
        }}
    </style>
</head>
<body>
    <div class="container">
        <!-- Company Logo -->
        <img src="https://resource.esriuk.com/wp-content/uploads/2017/06/GLA-Logo-Resized.png" alt="Company Logo" class="logo">

        <!-- Report Title -->
        <h1>Population Projections Outlier Report</h1>

        <!-- Add Check Results -->
        {check_results}

        <!-- Tallied Dataframe -->
        <h2>Tallied Outlier Table</h2>
        <div class="table-container">
            {tally_df_html}
        </div>

        <!-- Average Household Size Absolute Table -->
        <h2>Average Household Size Absolute Table</h2>
        <div class="table-container">
            {average_household_size_absolute_html}
        </div>

        <!-- Average Household Size Percentage Change Table -->
        <h2>Average Household Size Percentage Change Table</h2>
        <div class="table-container">
            {anomalous_average_household_size_pct_change_df_html}
        </div>

        <!-- Swarm plot -->
        <h2> Ward Distributions </h2>
        <div>
            <iframe src="http://localhost:2222" width="100%" height="800" frameborder="0"></iframe>
        </div>

        <!-- Population Pyramid Visualisation -->
        <h2>Interactive Population Pyramid App</h2>
        <div>
            <iframe src="http://localhost:1223" width="100%" height="800" frameborder="0"></iframe>
        </div>

         <!-- Population Pyramid Visualisation -->
        <h2>Component trend over time</h2>
        <div>
            <iframe src="http://localhost:1333" width="100%" height="800" frameborder="0"></iframe>
        </div>

        <!-- Interactive Plot -->
        <h2>Population project by component</h2>
        <div>
            <iframe src="interactive_line_plot.html" width="100%" height="600" frameborder="0"></iframe>
        </div>

        <!-- Footer Section -->
        <div class="footer">
            <p>This is an automated report produced by the Greater London Authority (GLA). The script provided is merely a suggestion for areas to investigate further regarding outliers and potential errors in the population projections. It is not guaranteed to capture every error that may exist within the dataset.</p>
            <p>If you require further information, please email <a href="mailto:Sebastian.Heslin-Rees@london.gov.uk">Sebastian.Heslin-Rees@london.gov.uk</a>.</p>
        </div>
    </div>
</body>
</html>
"""

# Save the report to an HTML file
with open("population_projections_report_with_checks.html", "w") as f:
    f.write(html_content)

print("HTML report with checks generated successfully!")

